In [4]:
import pandas as pd
import praw
from datetime import datetime
from time import sleep
from dotenv import load_dotenv
import os

load_dotenv()

# Authentication details from Reddit Developer Console
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT")

# Initialize PRAW Reddit instance
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)

print(f"Logged in as: {reddit.user.me()}")  # Verify connection: output should be "None" for read-only access

Logged in as: None


In [5]:
# Testing Purpose
for post in reddit.subreddit("UIUC").new(limit=1):
    # post.comments.replace_more(limit=0)
    # [print(comment.body) for comment in post.comments.list()[:10]]
    # print(post.comments.list().body)
    print(datetime.fromtimestamp(post.created_utc).isoformat())

2024-12-03T15:32:44


In [10]:
subreddit_name = "UIUC"
subreddit = reddit.subreddit(subreddit_name)

limit = 100  # Number of posts to scrape
posts = []

for post in subreddit.new(limit=limit):  # Change 'hot' to 'new', 'top', etc., as needed
    post.comments.replace_more(limit=0)

    posts.append({
        "title": post.title,
        "author": post.author.name if post.author else "deleted",
        "upvote": post.score,
        "body": post.selftext if post.selftext else "",
        "comments": [
            {
                "author": comment.author.name if comment.author else "deleted",
                "body": comment.body if comment.body else "",
                "created_at": datetime.fromtimestamp(comment.created_utc).isoformat()
            } for comment in post.comments.list()],
        "id": post.name,
        "url": post.url,
        "permalink": post.permalink,
        "created_at": datetime.fromtimestamp(post.created_utc).isoformat(),
    })
    sleep(1)

df = pd.DataFrame(posts)
df.to_csv("uiuc_subreddit_posts.csv", index=False)
print("posts saved to uiuc_subreddit_posts.csv")

posts saved to uiuc_subreddit_posts.csv


In [54]:
df = pd.read_csv("uiuc_subreddit_posts.csv")
pd.set_option('display.max_colwidth', None)
df.shape

(100, 9)

In [55]:
df.head()

,title,author,upvote,body,comments,id,url,permalink,created_at
0,What’s the workload in PHIL 104 - ACP like?,Celestial1007,0,Title. Thanks!,[],t3_1gudjjx,https://www.reddit.com/r/UIUC/comments/1gudjjx/whats_the_workload_in_phil_104_acp_like/,/r/UIUC/comments/1gudjjx/whats_the_workload_in_phil_104_acp_like/,2024-11-18T13:42:54
1,Physics 211 Scheduling Question,FelixTheNoob27,0,I’m making my schedule and it’s a bit of a struggle with availability so I might end up with a discussion/lab before a Tuesday lecture but is this still ok because I feel I’m going into a discussion before lecture and I might not be up to date. How does this work?\nThanks,[],t3_1gud92r,https://www.reddit.com/r/UIUC/comments/1gud92r/physics_211_scheduling_question/,/r/UIUC/comments/1gud92r/physics_211_scheduling_question/,2024-11-18T13:30:43
2,Subleasing Spring Semester,FaithlessnessDear73,2,Hi there! I am subleasing my room at 211 E John St in Champaign for the Spring semester. Please let me know if interested\n\nDetails: \n- 4 bed / 2 bath\n- $622 rent + utilities\n- Next to Green St\n- 10 min walk from Quad\n\nYou’ll be living with 3 girls but I promise they’re awesome (open to any gender),[],t3_1gud6qv,https://www.reddit.com/gallery/1gud6qv,/r/UIUC/comments/1gud6qv/subleasing_spring_semester/,2024-11-18T13:28:03
3,Illinois included in Nine Colleges with Best Financial Planning Programs!!!,proflem,1,NaN,"[{'author': 'proflem', 'body': 'Our students and alumni earned this - #7 out of 179 programs nationwide feels amazing. This is our first time making this list of top programs. \n \nYou can learn more about our program at: [https://publish.illinois.edu/financial-planning/](https://publish.illinois.edu/financial-planning/)', 'created_at': '2024-11-18T12:55:33'}]",t3_1gucbbz,https://www.wealthmanagement.com/careers/nine-colleges-best-financial-planning-programs,/r/UIUC/comments/1gucbbz/illinois_included_in_nine_colleges_with_best/,2024-11-18T12:53:11
4,Color appointments available this week :),PinkLemonadeHair,0,Book here: https://linktr.ee/pink.lemonade.hair or text/call 872-269-4472\n,[],t3_1gubzq8,https://i.redd.it/hlde2o73gp1e1.jpeg,/r/UIUC/comments/1gubzq8/color_appointments_available_this_week/,2024-11-18T12:40:22
